# Final project

[References]

[Version]

[Revision history]
* 2022-04-24: Preprocessing 일부 진행 및 추천 알고리즘 탐색

## 0. Mounting driver

In [1]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 1. Preprocessing

In [ ]:
import pandas as pd
import json
import gc # 메모리 대비

In [ ]:
with open('/content/drive/Shareddrives/BKMS/BKMS_FinalProject/Data/part1.json', encoding="utf-8") as f:
  data = json.load(f)

In [ ]:
title_lst = []
key_lst = []
citeNum_lst=[]

In [ ]:
# Create 'Title' DataFrame
for i in data :
    json_title = i['title']
    title_lst.append(json_title)

title_df = pd.DataFrame(title_lst, columns = ['Title'])

In [ ]:
# Create 'Keywords' DataFrame
for i in data :
    json_key = i['keywords']
    key_lst.append(json_key)

In [ ]:
keys = ['key1', 'key2', 'key3', 'key4', 'key5', 'key6', 'key7', 'key8', 'key9', 'key10', 'key11', 'key12', 'key13', 'key14', 'key15',
       'key16', 'key17', 'key18', 'key19', 'key20', 'key21', 'key22', 'key23', 'key24', 'key25', 'key26', 'key27', 'key28', 'key29',
       'key30', 'key31', 'key32', 'key33']
key_df = pd.DataFrame(key_lst, columns = keys)
key_df['Keywords'] =key_df[keys].apply(lambda row: ':'.join(row.values.astype(str)), axis=1)

In [ ]:
# Create 'n_citation' DataFrame
for i in data:
    if 'n_citation' in i:
        json_ncite = i['n_citation']
    else:
        json_ncite = 0
    citeNum_lst.append(json_ncite)

citeNum_df = pd.DataFrame(citeNum_lst, columns = ['Citation number'])

In [ ]:
title_df.head()

,Title
0,Timing yield estimation using statistical stat...
1,34957+70764=105621
2,3GIO.
3,1may
4,300


In [ ]:
key_df.head()

,key1,key2,key3,key4,key5,key6,key7,key8,key9,key10,...,key25,key26,key27,key28,key29,key30,key31,key32,key33,Keywords
0,sequential circuits,statistical distributions,set-up time constraints,register-to-register paths,statistical static timing analysis,integrated circuit modelling,parameter estimation,statistical analysis,circuit model,path delays,...,monte carlo simulation,design method,static timing analysis,design methodology,process variation,shortest path,registers,circuit design,circuit analysis,sequential circuits:statistical distributions:...
1,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None:None:None:None:None:None:None:None:None:N...
2,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None:None:None:None:None:None:None:None:None:N...
3,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None:None:None:None:None:None:None:None:None:N...
4,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None:None:None:None:None:None:None:None:None:N...


In [ ]:
citeNum_df.head()

,Citation number
0,28
1,0
2,0
3,0
4,0


## 2. Algorithm

### 1) Content-based

In [ ]:
# Content-based

from sklearn.metrics.pairwise import cosine_similarity

def top_k_items(item_id, top_k, corr_mat, map_name):
    
    # sort correlation value ascendingly and select top_k item_id
    top_items = corr_mat[item_id,:].argsort()[-top_k:][::-1] 
    top_items = [map_name[e] for e in top_items] 

    return top_items

# preprocessing
rated_items = items.loc[items[ITEM_COL].isin(ratings[ITEM_COL])].copy()

# extract the genre
genre = rated_items['genres'].str.split(",", expand=True)

# get all possible genre
all_genre = set()
for c in genre.columns:
    distinct_genre = genre[c].str.lower().str.strip().unique()
    all_genre.update(distinct_genre)
all_genre.remove(None)

# create item-genre matrix
item_genre_mat = rated_items[[ITEM_COL, 'genres']].copy()
item_genre_mat['genres'] = item_genre_mat['genres'].str.lower().str.strip()

# OHE the genres column
for genre in all_genre:
    item_genre_mat[genre] = np.where(item_genre_mat['genres'].str.contains(genre), 1, 0)
item_genre_mat = item_genre_mat.drop(['genres'], axis=1)
item_genre_mat = item_genre_mat.set_index(ITEM_COL)

# compute similarity matix
corr_mat = cosine_similarity(item_genre_mat)

# get top-k similar items
ind2name = {ind:name for ind,name in enumerate(item_genre_mat.index)}
name2ind = {v:k for k,v in ind2name.items()}
similar_items = top_k_items(name2ind['99'],
                            top_k = 10,
                            corr_mat = corr_mat,
                            map_name = ind2name)

# display result
print("The top-k similar movie to item_id 99")
display(items.loc[items[ITEM_COL].isin(similar_items)])

del corr_mat
gc.collect();

### 2) Deep learning MF - Neural Collaborative Filtering

In [ ]:
!pip install recommenders
!pip install tf-slim
from recommenders.models.ncf.ncf_singlenode import NCF
from recommenders.models.ncf.dataset import Dataset as NCFDataset
from recommenders.datasets.python_splitters import python_chrono_split
from recommenders.utils.constants import SEED as DEFAULT_SEED

In [ ]:
# Initial parameters
TOP_K = 10
EPOCHS = 50
BATCH_SIZE = 1024
SEED = DEFAULT_SEED
# default column name
ratings.columns = ["userID","itemID","rating"]
ratings["timestamp"] = pd.to_datetime('2022-01-23') + pd.to_timedelta(np.random.randint(-100,0,size=(len(ratings),)), unit='d')
train, test = python_chrono_split(ratings, 0.75)
data = NCFDataset(train = train, test = test, seed=SEED,col_timestamp=None)
model = NCF (
    n_users=data.n_users, 
    n_items=data.n_items,
    model_type="NeuMF",
    n_factors=4,
    layer_sizes=[16,8,4],
    n_epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=1e-3,
    verbose=1,
    seed=SEED
)

# fitting the model
model.fit(data)

# predict the data in the test set
predictions = [[row.userID, row.itemID, model.predict(row.userID, row.itemID)]
               for (_, row) in test.iterrows()]
               
item_input = np.array([model.item2id[i] for i in test["itemID"].values.tolist()])
item_emb = model.sess.run( model.embedding_mlp_Q   , feed_dict = {model.item_input : item_input[..., None] })

item_corr_mat = cosine_similarity(item_emb)

print("\nThe top-k similar movie to item_id 99")
similar_items = top_k_items(name2ind['99'],
                            top_k = 10,
                            corr_mat = item_corr_mat,
                            map_name = ind2name)

display(items.loc[items[ITEM_COL].isin(similar_items)])